## 준비하기
1. 아래 예제를 실행하기 위해서는 사전에 ./setup.sh 파일을 실행하거나 ./start_notebook.sh 로 노트북을 구동하여 의존성 라이브러리들을 미리 설치해두셔야 합니다.
2. qdrant서버가 떠 있어야 정상 동작합니다. ./start_qdrant.sh 를 실행하여 qdrant 서버를 띄워주세요.
3. openapi 키가 있어야 합니다. 

## 목적:
- LangChain RAG 체인을 활용하여 벡터 기반 질의응답 시스템 구축

## 왜 필요한가?
- 벡터 기반 검색을 사용하여 LLM의 답변 정확도를 높일 수 있음
- 기존 RAG 시스템과 LangChain을 쉽게 결합 가능

## 주요 개념:
- **RAG (Retrieval-Augmented Generation)**: 검색과 생성 모델을 결합하여 답변을 개선하는 기술
- **LangChain**: LLM과 외부 데이터베이스(Qdrant 등)를 쉽게 연결하는 프레임워크

In [2]:
# 라이브러리 로딩 (아래 부분에서 오류가 발생하면 pip -r ./requirements.txt 로 의존성을 설치해주세요)
import os
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore

In [3]:
# Qdrant 연결
qdrant_client = QdrantClient(host="localhost", port=6333)
collection_name = "sample_vectors"

# Qdrant 컬렉션 생성 (없을 경우 자동 생성)
if not qdrant_client.collection_exists(collection_name):
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=384, distance=Distance.COSINE)  # 384는 임베딩 차원 수
    )

In [4]:
# 임베딩 모델 로드
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [5]:
# Qdrant 기반 벡터 저장소 생성
vectorstore = QdrantVectorStore(
    client=qdrant_client,
    collection_name=collection_name,
    embedding=embeddings
)

In [6]:
# OpenAI API 키 설정 (환경 변수에서 가져오거나 기본값 설정)
openai_api_key = os.environ.get("OPENAI_API_KEY", "your_api_key")

# LLM 설정 (GPT API)
llm = ChatOpenAI(api_key=openai_api_key)

In [7]:
# 시스템 프롬프트 설정
system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentences maximum and keep the answer concise. "
    "Context: {context}"
)

In [8]:
# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# 문서 결합 체인 생성
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

# RAG 체인 생성
retriever = vectorstore.as_retriever()
qa_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [9]:
# 질문 실행 및 응답 출력
question = "What is the purpose of vector databases?"
response = qa_chain.invoke({"input": question})
print("LLM 응답:", response["answer"])

LLM 응답: Vector databases are used to store and analyze geographical data, as they are optimized for spatial data and operations. They allow for efficient querying, storage, and retrieval of complex spatial information such as points, lines, and polygons. Vector databases enable applications like geographic information systems (GIS) to handle and process spatial data effectively.
